<a href="https://colab.research.google.com/github/Edenshmuel/PapaJohns_Data_Science_Project/blob/main/Temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Final_Project_PapaJohns/All_years.csv')

<ipython-input-1-44878571b353>:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Final_Project_PapaJohns/All_years.csv')


In [2]:
df

,Date,Month,סוג פריט,תאור סוג,מס' פריט,תאור פריט,הזמנה,כמות,סכום,הנחה לפריט,אחרי הנחה,Is_original
0,2022-01-01,1,0.0,NaN,67,NaN,14328.0,1,4.95,0.0,4.95,1
1,2022-01-01,1,0.0,NaN,67,NaN,14354.0,2,9.90,0.0,9.90,1
2,2022-01-01,1,1.0,פיצות מיוחדות 14',4,ביאנקה 14' [],14321.0,1,85.90,0.0,85.90,1
3,2022-01-01,1,1.0,פיצות מיוחדות 14',7,טוסקנית 14' [],14304.0,1,85.90,0.0,85.90,1
4,2022-01-01,1,1.0,פיצות מיוחדות 14',13,המומלצת 14' [],14358.0,1,89.90,0.0,89.90,1
...,...,...,...,...,...,...,...,...,...,...,...,...
258189,2024-12-31,12,52,מנות נלוות,3117,שובר 50 שח DV,7669633.0,5,125.00,0.0,125.00,1
258190,2024-12-31,12,52,מנות נלוות,3210,2 משפחתיות קלאסיות + 2 תוספות צמחוניות חבר,7669582.0,1,0.00,0.0,0.00,1
258191,2024-12-31,12,52,מנות נלוות,3210,2 משפחתיות קלאסיות + 2 תוספות צמחוניות חבר,7669585.0,1,19.90,0.0,19.90,1
258192,2024-12-31,12,52,מנות נלוות,3210,2 משפחתיות קלאסיות + 2 תוספות צמחוניות חבר,7669605.0,1,19.90,0.0,19.90,1


In [3]:
from google.colab import files
uploaded = files.upload()

Saving preprocessing.py to preprocessing.py


In [5]:
!pip install convertdate

In [6]:
!pip install fuzzywuzzy

In [4]:
# אם הקובץ נקרא preprocessing.py
import preprocessing

# טען את הפונקציה
from preprocessing import prepare_data

In [5]:
df_ready = prepare_data(df)

In [6]:
df_ready.tail(10)

,Date,Year,Month,Day,Day_Name,Is_Weekend,Cleaned Description,Category,Quantity,Christian_Holiday,...,Day_Name_Cos,Month_Sin,Month_Cos,Avg_Quantity,Std_Quantity,Num_Days_Sold,Popularity_Score,Jewish_Holiday_Encoded,Christian_Holiday_Encoded,Portion_Type_Encoded
250763,2024-12-31,2024,12,31,3,0,תירס,Topping,1,False,...,-0.900969,-2.449294e-16,1.0,1.308601,0.649768,648,1.308601,0,12,1
250764,2024-12-31,2024,12,31,3,0,תירס,Topping,1,False,...,-0.900969,-2.449294e-16,1.0,1.308601,0.649768,648,1.308601,0,12,1
250765,2024-12-31,2024,12,31,3,0,זיתים ירוקים,Topping,1,False,...,-0.900969,-2.449294e-16,1.0,1.433545,0.763264,653,1.433545,0,12,1
250766,2024-12-31,2024,12,31,3,0,זיתים ירוקים,Topping,1,False,...,-0.900969,-2.449294e-16,1.0,1.433545,0.763264,653,1.433545,0,12,1
250767,2024-12-31,2024,12,31,3,0,זיתים ירוקים,Topping,2,False,...,-0.900969,-2.449294e-16,1.0,1.433545,0.763264,653,1.433545,0,12,1
250768,2024-12-31,2024,12,31,3,0,זיתים ירוקים,Topping,1,False,...,-0.900969,-2.449294e-16,1.0,1.433545,0.763264,653,1.433545,0,12,1
250769,2024-12-31,2024,12,31,3,0,זיתים ירוקים,Topping,2,False,...,-0.900969,-2.449294e-16,1.0,1.433545,0.763264,653,1.433545,0,12,1
250770,2024-12-31,2024,12,31,3,0,זיתים ירוקים,Topping,2,False,...,-0.900969,-2.449294e-16,1.0,1.433545,0.763264,653,1.433545,0,12,1
250749,2024-12-31,2024,12,31,3,0,בצל סגול,Topping,2,False,...,-0.900969,-2.449294e-16,1.0,1.374919,0.563636,638,1.374919,0,12,1
251240,2024-12-31,2024,12,31,3,0,משפחתיות קלאסיות תוספות צמחוניות חבר,Topping,2,False,...,-0.900969,-2.449294e-16,1.0,3.000000,1.001354,31,3.000000,0,12,2


In [8]:
df_ready[
    df_ready["Cleaned Description"].str.contains('טונה דה לקס ', na=False)
][["Cleaned Description", "Category", "Quantity"]]


,Cleaned Description,Category,Quantity
1694,טונה דה לקס דקה 16,Topping,1
1693,טונה דה לקס דקה 16,Main Dish,1
6270,טונה דה לקס דקה 14,Topping,1
6269,טונה דה לקס דקה 14,Main Dish,1
8450,טונה דה לקס דקה 16,Topping,1
8449,טונה דה לקס דקה 16,Main Dish,1
9145,טונה דה לקס דקה 16,Topping,1
9144,טונה דה לקס דקה 16,Main Dish,1
9578,טונה דה לקס דקה 16,Topping,1
9577,טונה דה לקס דקה 16,Main Dish,1


In [9]:
df_ready["Cleaned Description"].unique()

array(['ביאנקה', 'ארוחת שחיתות', 'פיצות אישיות', 'פאפא דיל',
       'מבצע ב 19 שח', 'מבצע פיצה קלאסית ב649 שח', 'קלאסית',
       'פיצות משפחתיות', 'פיצה אישית', '3 מרגריטות קלאסיות',
       'קלאסית ושתייה', 'זיתי קלמטה', 'פפרוני טבעוני', 'עגבניות',
       'בולגרית', 'חלפיניו', 'פיצות תוספות', 'רוטב באפלו קטן',
       'פיצה ללא גלוטן 14', 'קלאסית טבעונית איטסיין', 'טבעונית 8',
       'ציזי רול טבעוני', 'טבעונית 14', 'היוונית איטסיין',
       'קלאסית איטסיין', 'פיצה אישי 8', 'רטבים לבחירה', 'קלאסית שתייה',
       'שוקולד קראנץ רול', '2 מרגריטות קלאסיות', 'קלאסית איטסיין במקום',
       'רוטב פיצה מארז', 'רוטב שום פאפא מארז', 'רוטב באפלו מארז',
       'רוטב ברביקיו מארז', 'פאפא מיקס דקה 16', 'היוונית דקה 16',
       'היוונית', 'קלאסית דקה 14', 'טוסקנית', 'הצמחונית', 'קלאסית דקה 16',
       'בצל לבן', 'רוטב פיצה קטן', 'פחית ספרייט זירו', 'פחית קולה זירו',
       'פחית קוקה קולה', 'מים מינרלים נביעות 500 מל',
       'פיוז טי אפרסק 500 מל', 'ספרייט5 ליטר', 'ספרייט זירו5 ליטר',
       'פיוז טי

In [14]:
df_ready.to_csv("cleaned_data.csv", index=False)